In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# 1. 원본 데이터 파일
INPUT_CSV = 'final_802_data_for_training.csv'
TARGET_COLUMN = 'tsunami' # 타겟(y) 컬럼 이름

# 2. ★★★ 예측 스크립트와 호환되는 6개 특성 ★★★
FEATURE_COLUMNS = [
    'magnitude',
    'depth',
    'is_ocean',
    'is_steep_slope',
    'horizontal_count_1y_full', # (h_count)
    'vertical_count_1y_full'    # (v_count)
]

# 3. 저장할 파일 이름
SCALER_OUTPUT = 'tsunami_scaler.joblib' # (스케일러 모델 파일)
SCALED_DATA_OUTPUT = 'scaled_model_data_6features.csv' # (정형화된 데이터 파일)

def create_scaler_and_scaled_data():
    """
    (수정됨) 6-feature 스케일러를 생성하고, 스케일러(.joblib)와
    정형화된 데이터(.csv)를 모두 저장합니다.
    """
    try:
        # 원본 데이터 로드
        df = pd.read_csv(INPUT_CSV)
        print(f"'{INPUT_CSV}' 로드 완료.")

        # 6개 특성(X)과 타겟(y) 분리
        X_features = df[FEATURE_COLUMNS]
        y_target = df[TARGET_COLUMN]

        print(f"예측 호환용 6개 특성 선택 완료: {FEATURE_COLUMNS}")

        # StandardScaler 생성 및 학습(fit)
        scaler = StandardScaler()
        scaler.fit(X_features)

        print("StandardScaler 학습(fit) 완료.")

        # --- 작업 1: 스케일러(모델) 파일 저장 ---
        joblib.dump(scaler, SCALER_OUTPUT)
        print(f"\n[작업 1] 성공! 스케일러가 '{SCALER_OUTPUT}' 파일로 저장되었습니다.")


        # --- 작업 2: 정형화된 데이터 파일 저장 (사용자 요청 추가) ---
        print(f"\n[작업 2] 데이터를 변환하여 '{SCALED_DATA_OUTPUT}' 파일로 저장합니다...")

        # 1. 스케일러로 6개 특성 데이터를 변환(transform)
        X_scaled = scaler.transform(X_features)

        # 2. 변환된 NumPy 배열을 다시 DataFrame으로 만듦 (컬럼명 복원)
        df_scaled_features = pd.DataFrame(X_scaled, columns=FEATURE_COLUMNS)

        # 3. 타겟(tsunami) 열을 다시 결합
        df_final_scaled_data = df_scaled_features.copy()
        df_final_scaled_data[TARGET_COLUMN] = y_target.values

        # 4. 새 CSV 파일로 저장
        df_final_scaled_data.to_csv(SCALED_DATA_OUTPUT, index=False)

        print(f"성공! 정형화된 데이터(6-features + target)가 '{SCALED_DATA_OUTPUT}' 파일로 저장되었습니다.")

        print("\n이제 'train_v18_model.py' 스크립트를 실행하여 모델을 훈련할 수 있습니다.")

    except FileNotFoundError:
        print(f"오류: '{INPUT_CSV}' 파일을 찾을 수 없습니다.")
    except KeyError as e:
        print(f"오류: '{INPUT_CSV}' 파일에 필요한 특성 열이 없습니다: {e}")
        print(f"필요한 열: {FEATURE_COLUMNS} 또는 '{TARGET_COLUMN}'")
    except Exception as e:
        print(f"작업 중 오류 발생: {e}")

if __name__ == "__main__":
    create_scaler_and_scaled_data()

'final_802_data_for_training.csv' 로드 완료.
예측 호환용 6개 특성 선택 완료: ['magnitude', 'depth', 'is_ocean', 'is_steep_slope', 'horizontal_count_1y_full', 'vertical_count_1y_full']
StandardScaler 학습(fit) 완료.

[작업 1] 성공! 스케일러가 'tsunami_scaler.joblib' 파일로 저장되었습니다.

[작업 2] 데이터를 변환하여 'scaled_model_data_6features.csv' 파일로 저장합니다...
성공! 정형화된 데이터(6-features + target)가 'scaled_model_data_6features.csv' 파일로 저장되었습니다.

이제 'train_v18_model.py' 스크립트를 실행하여 모델을 훈련할 수 있습니다.


In [19]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import joblib
import warnings

# 경고 메시지 무시
warnings.filterwarnings('ignore')

def train_model_from_preprocessed(input_file, model_output_file):
    """
    이미 전처리된 데이터를 로드하여 XGBoost 모델을 훈련시키고,
    성능을 평가한 뒤 모델을 저장합니다.
    """
    try:
        # 1. 전처리된 데이터 로드
        df = pd.read_csv(input_file)
        print(f"전처리된 파일 '{input_file}' 로드 완료. (총 {len(df)}개 행)")

        # 2. 피처(X)와 타겟(y) 분리
        target_column = 'tsunami'
        if target_column not in df.columns:
            print(f"오류: 타겟 컬럼 '{target_column}'을 찾을 수 없습니다.")
            return

        X = df.drop(target_column, axis=1)
        y = df[target_column]
        print("피처(X)와 타겟(y) 분리 완료.")

        # 3. 학습 데이터와 테스트 데이터 분리 (80% 학습, 20% 테스트)
        # stratify=y : 원본 데이터의 타겟(tsunami) 비율을 유지하며 분리
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,  # 스케일링된 X를 바로 사용
            test_size=0.2,
            random_state=42,
            stratify=y
        )
        print(f"데이터 분리 완료: 학습용 {len(X_train)}개, 테스트용 {len(X_test)}개")

        # 4. XGBoost 모델 초기화 및 학습
        # scale_pos_weight: 타겟(y)의 0과 1 비율 불균형을 보정합니다.
        # (0의 개수 / 1의 개수)
        ratio = (y == 0).sum() / (y == 1).sum()

        model = xgb.XGBClassifier(
            objective='binary:logistic',  # 이진 분류
            scale_pos_weight=ratio,       # 클래스 불균형 보정
            eval_metric='logloss',
            random_state=42,
            use_label_encoder=False # 최신 XGBoost 호환성
        )

        print("\nXGBoost 모델 학습 시작...")
        model.fit(X_train, y_train)
        print("모델 학습 완료.")

        # 5. 모델 성능 평가 (테스트 데이터)
        y_pred = model.predict(X_test)

        # 정확도 (Accuracy)
        accuracy = accuracy_score(y_test, y_pred)

        # 정밀도 (Precision): 모델이 '쓰나미 발생(1)'이라고 예측한 것 중, '실제 쓰나미(1)'인 비율
        precision = precision_score(y_test, y_pred)

        print("\n--- 모델 성능 평가 (Test Set) ---")
        print(f"정확도 (Accuracy):   {accuracy * 100:.2f} %")
        print(f"정밀도 (Precision): {precision * 100:.2f} %")
        print("---------------------------------")

        # 6. 훈련된 모델 파일로 저장
        joblib.dump(model, model_output_file)
        print(f"\n훈련된 XGBoost 모델 저장 완료: '{model_output_file}'")

    except FileNotFoundError:
        print(f"오류: '{input_file}' 파일을 찾을 수 없습니다.")
        print("이 스크립트는 'scaled_final_model_data.csv' 파일이 필요합니다.")
    except Exception as e:
        print(f"작업 중 오류 발생: {e}")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    # [중요] 입력 파일로 'scaled_final_model_data.csv'을 사용합니다.
    INPUT_CSV = 'scaled_model_data_6features.csv'
    MODEL_OUTPUT = 'xgboost_tsunami_model.joblib'

    train_model_from_preprocessed(INPUT_CSV, MODEL_OUTPUT)

전처리된 파일 'scaled_model_data_6features.csv' 로드 완료. (총 802개 행)
피처(X)와 타겟(y) 분리 완료.
데이터 분리 완료: 학습용 641개, 테스트용 161개

XGBoost 모델 학습 시작...
모델 학습 완료.

--- 모델 성능 평가 (Test Set) ---
정확도 (Accuracy):   60.25 %
정밀도 (Precision): 50.82 %
---------------------------------

훈련된 XGBoost 모델 저장 완료: 'xgboost_tsunami_model.joblib'


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# 1. 원본 데이터 파일
INPUT_CSV = 'final_model_dataset.csv'

# 2. ★★★ 예측 스크립트와 호환되는 6개 특성 ★★★
# (예측 스크립트의 X_new = np.array([...]) 순서와 정확히 일치해야 함)
FEATURE_COLUMNS = [
    'magnitude',
    'depth',
    'is_ocean',
    'is_steep_slope',
    'horizontal_count_1y_full', # (h_count)
    'vertical_count_1y_full'    # (v_count)
]

# 3. 예측 스크립트가 요구하는 스케일러 파일 이름
SCALER_OUTPUT = 'tsunami_scaler.joblib'

def create_compatible_scaler():
    """
    예측 스크립트(V18)와 호환되는 6-feature 스케일러를 생성하고 저장합니다.
    """
    try:
        # 원본 데이터 로드
        df = pd.read_csv(INPUT_CSV)
        print(f"'{INPUT_CSV}' 로드 완료.")

        # 6개 특성만 선택
        X_features = df[FEATURE_COLUMNS]
        print(f"예측 호환용 6개 특성 선택 완료: {FEATURE_COLUMNS}")

        # StandardScaler 생성 및 학습(fit)
        scaler = StandardScaler()
        scaler.fit(X_features)

        print("StandardScaler 학습(fit) 완료.")

        # 학습된 스케일러 객체를 파일로 저장
        joblib.dump(scaler, SCALER_OUTPUT)

        print(f"\n성공! '{SCALER_OUTPUT}' 파일이 저장되었습니다.")
        print("이제 'train_v18_model.py' 스크립트를 실행하세요.")

    except FileNotFoundError:
        print(f"오류: '{INPUT_CSV}' 파일을 찾을 수 없습니다.")
    except KeyError:
        print(f"오류: '{INPUT_CSV}' 파일에 필요한 특성 열이 없습니다.")
        print(f"필요한 열: {FEATURE_COLUMNS}")
    except Exception as e:
        print(f"작업 중 오류 발생: {e}")

if __name__ == "__main__":
    create_compatible_scaler()

'final_model_dataset.csv' 로드 완료.
예측 호환용 6개 특성 선택 완료: ['magnitude', 'depth', 'is_ocean', 'is_steep_slope', 'horizontal_count_1y_full', 'vertical_count_1y_full']
StandardScaler 학습(fit) 완료.

성공! 'tsunami_scaler.joblib' 파일이 저장되었습니다.
이제 'train_v18_model.py' 스크립트를 실행하세요.


In [13]:
import pandas as pd

# --- 설정 ---
INPUT_FILE = 'final_model_data.csv'
OUTPUT_FILE = 'final_model_data_6features.csv'

# 제거할 컬럼 목록
COLUMNS_TO_DROP = [
    'latitude',
    'longitude',
    'Year',
    'Month'
]

# (참고) 최종적으로 남게 될 컬럼 목록
# Features: 'magnitude', 'depth', 'is_ocean', 'is_steep_slope',
#           'horizontal_count_1y_full', 'vertical_count_1y_full'
# Target: 'tsunami'

def extract_6_features(input_file, output_file, columns_to_drop):
    """
    원본 데이터 파일에서 지정된 컬럼들을 제거하고 새 파일로 저장합니다.
    """
    try:
        # 1. 원본 데이터 로드
        df = pd.read_csv(input_file)
        print(f"'{input_file}' 로드 완료. (원본 컬럼: {list(df.columns)})")

        # 2. 지정된 컬럼 제거
        # df.drop()은 컬럼이 없는 경우 오류를 발생시킬 수 있으므로,
        # 현재 데이터프레임에 '있는' 컬럼만 제거하도록 처리합니다.

        existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

        if not existing_columns_to_drop:
            print(f"제거할 컬럼이 '{input_file}'에 존재하지 않습니다.")
            return

        df_reduced = df.drop(columns=existing_columns_to_drop)

        print(f"컬럼 제거 완료: {existing_columns_to_drop}")

        # 3. 새 파일로 저장
        df_reduced.to_csv(output_file, index=False)

        print(f"\n성공! 6-feature 데이터셋이 '{output_file}'로 저장되었습니다.")
        print(f"저장된 컬럼: {list(df_reduced.columns)}")
        print("\n이제 이 파일을 사용하여 '스케일러 생성' 및 '모델 훈련'을 진행할 수 있습니다.")

    except FileNotFoundError:
        print(f"오류: '{input_file}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"작업 중 오류 발생: {e}")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    extract_6_features(INPUT_FILE, OUTPUT_FILE, COLUMNS_TO_DROP)

'final_model_data.csv' 로드 완료. (원본 컬럼: ['magnitude', 'depth', 'latitude', 'longitude', 'Year', 'Month', 'tsunami', 'is_ocean', 'is_steep_slope', 'horizontal_count_1y_full', 'vertical_count_1y_full'])
컬럼 제거 완료: ['latitude', 'longitude', 'Year', 'Month']

성공! 6-feature 데이터셋이 'final_model_data_6features.csv'로 저장되었습니다.
저장된 컬럼: ['magnitude', 'depth', 'tsunami', 'is_ocean', 'is_steep_slope', 'horizontal_count_1y_full', 'vertical_count_1y_full']

이제 이 파일을 사용하여 '스케일러 생성' 및 '모델 훈련'을 진행할 수 있습니다.
